In [61]:
import cv2
import mediapipe as mp
import numpy as np

# Cargar imagen del emoji de hamburguesa y del corazon
burger_img = cv2.imread('images/burger_emoji.png', cv2.IMREAD_UNCHANGED)
heart_img = cv2.imread('images/heart_emoji.png', cv2.IMREAD_UNCHANGED)
hat_img = cv2.imread('images/hat_img.png', cv2.IMREAD_UNCHANGED)

burger_positions = []

def draw_burgers(distance, lip_top, frame):
    frame_height, frame_width = frame.shape[0], frame.shape[1]
    
    if distance > 70:  # Umbral
        print(f"Distancia boca abierta: {distance}")
        
        burger_resized = cv2.resize(burger_img, (50, 50))
        burger_width, burger_height = burger_resized.shape[1], burger_resized.shape[0]

        # Generar nuevas hamburguesas en la parte derecha e izquierda de la cara
        for i in range(5):  # Número de hamburguesas a mostrar
            if i >= len(burger_positions):
                # Inicializar la posición de las hamburguesas al estilo json
                burger_positions.append({
                    "x": lip_top[0] + 100 + (i * 80),  # Espaciado entre hamburguesas en el lado derecho
                    "y": -burger_height,               # Comenzar desde la parte superior fuera de la vista
                    "direction": 1,                    # Dirección hacia abajo
                    "side": "right"                    # Indicar que es del lado derecho
                })
                burger_positions.append({
                    "x": lip_top[0] - 100 - (i * 80),  # Espaciado entre hamburguesas en el lado izquierdo
                    "y": -burger_height,               # Comenzar desde la parte superior fuera de la vista
                    "direction": 1,                    # Dirección hacia abajo
                    "side": "left"                     # Indicar que es del lado izquierdo
                })
            
            # Actualizamos la posición de las hamburguesas
            for j in range(len(burger_positions)):
                burger_positions[j]["y"] += burger_positions[j]["direction"] * 5  # Mover hacia abajo
                if burger_positions[j]["y"] > frame_height: burger_positions[j]["y"] = -burger_height  # Volver a la parte superior

                # Verificar que la hamburguesa no se salga del cuadro
                x_offset, y_offset  = burger_positions[j]["x"], burger_positions[j]["y"]

                if x_offset + burger_width <= frame_width and y_offset + burger_height <= frame_height:
                    # Dibujar la hamburguesa en su nueva posición
                    for c in range(0, 3):
                        try:
                            frame[y_offset:y_offset+burger_height, x_offset:x_offset+burger_width, c] = \
                                burger_resized[:, :, c] * (burger_resized[:, :, 3] / 255.0) + \
                                frame[y_offset:y_offset+burger_height, x_offset:x_offset+burger_width, c] * \
                                (1.0 - burger_resized[:, :, 3] / 255.0)
                        except Exception as e:
                            print(f"Error al dibujar hamburguesa: {e}")
                            
def apply_hat(frame, face_landmarks):
    # Utilizamos los puntos 183 y 332 de la malla facial como referencia para la parte superior de la cabeza
    head_top = (int(face_landmarks.landmark[183].x * frame.shape[1]), 
                int(face_landmarks.landmark[332].y * frame.shape[0]))

    # Combinación de puntos (punto 183 y punto 332).
    head_top_avg_y = int((face_landmarks.landmark[183].y + face_landmarks.landmark[332].y) * frame.shape[0] / 2)

    x_offset = head_top[0] - hat_img.shape[1] // 2       # Centrado horizontalmente
    y_offset = head_top_avg_y - (hat_img.shape[0] // 2)  # Ajuste hacia la parte superior de la cabeza
    
    # Constante de desplazamiento
    y_offset -= 150

    # Asegurarnos de que la gorra no quede fuera de la imagen
    frame_height, frame_width = frame.shape[:2]
    
    # Evitar que x_offset se salga por la izquierda y que y_offset se salga por la parte superior
    if x_offset < 0: x_offset = 0
    if y_offset < 0: y_offset = 0

    # Asegurarnos de que la gorra no se salga por la derecha ni por la parte inferior
    if x_offset + hat_img.shape[1] > frame_width: x_offset = frame_width - hat_img.shape[1]

    if y_offset + hat_img.shape[0] > frame_height: y_offset = frame_height - hat_img.shape[0]

    # Verificar que la gorra está dentro de los límites del frame
    if x_offset + hat_img.shape[1] <= frame_width and y_offset + hat_img.shape[0] <= frame_height:
        # Obtener la región de interés (ROI) en el frame
        roi = frame[y_offset:y_offset + hat_img.shape[0], x_offset:x_offset + hat_img.shape[1]]

        # Asegurar que la gorra tiene un canal alfa para manejar la transparencia
        for c in range(0, 3):
            roi[:, :, c] = roi[:, :, c] * (1.0 - hat_img[:, :, 3] / 255.0) + \
                            hat_img[:, :, c] * (hat_img[:, :, 3] / 255.0)

        # Colocar la gorra en el frame
        frame[y_offset:y_offset + hat_img.shape[0], x_offset:x_offset + hat_img.shape[1]] = roi
    else:
        print("La gorra se sale de los límites del frame.")


In [62]:
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)

def calculate_distance(p1, p2):
    # Calcular distancia euclidiana entre dos puntos (p1 y p2 son tuplas (x, y))
    return np.sqrt((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2)
           
# Función para obtener las coordenadas de los puntos clave de la malla facial
def process_frame(frame, face_mesh, points, mode):
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(frame_rgb)

    keypoints = []  # Lista para guardar las coordenadas de los puntos clave

    if results.multi_face_landmarks is not None:
        for face_landmarks in results.multi_face_landmarks:       
            # Dibujar y guardar los puntos específicos
            for point in points:
                if point < len(face_landmarks.landmark):  # Verificar índice válido
                    x = int(face_landmarks.landmark[point].x * frame.shape[1])
                    y = int(face_landmarks.landmark[point].y * frame.shape[0])
                    keypoints.append((x, y))  # Agregar las coordenadas a la lista
            if (mode == 2):
                apply_hat(frame, face_landmarks)
                
                # Calcular la distancia entre el labio superior e inferior (usando los puntos 13, 14, 18, 19)
                lip_top = (int(face_landmarks.landmark[13].x * frame.shape[1]), int(face_landmarks.landmark[14].y * frame.shape[0]))
                lip_bottom = (int(face_landmarks.landmark[18].x * frame.shape[1]), int(face_landmarks.landmark[19].y * frame.shape[0]))
                draw_burgers(calculate_distance(lip_top, lip_bottom), lip_top, frame)
    
    return keypoints


In [63]:
def aplicar_filtro(frame, mode, points):
    with mp_face_mesh.FaceMesh(
            static_image_mode=False,
            max_num_faces=1,
            min_detection_confidence=0.5) as face_mesh:
        keypoints = process_frame(frame, face_mesh, points, mode)
        return
    
def filtro_1(frame, mode):
    return aplicar_filtro(frame, mode, [0, 17, 468, 473])

def filtro_2(frame, mode):
    return aplicar_filtro(frame, mode, [127, 356, 183, 332])

def change_filter(frame, mode):
    if mode == 1: filtro_1(frame, mode)
    elif mode == 2: filtro_2(frame, mode)
    return

def main():
    cap = cv2.VideoCapture(0)  # Abrir la cámara
    mode = 1                   # Modo inicial
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
            
    if not cap.isOpened():
        print("No se puede abrir la cámara")
        exit()

    while True:
        ret, frame = cap.read()
        
        if not ret:
            break
        
        change_filter(frame, mode)
        # Muestrar el frame
        cv2.imshow('Frame', frame) 

        key = cv2.waitKey(1) & 0xFF
        if key == ord('1'): mode = 1
        elif key == ord('2'): mode = 2
        elif key == ord('q'): break
        
    cap.release()
    cv2.destroyAllWindows()

if __name__ == '__main__':
    main()

Distancia boca abierta: 70.3420215802759
Error al dibujar hamburguesa: operands could not be broadcast together with shapes (0,50) (50,50) 
Error al dibujar hamburguesa: operands could not be broadcast together with shapes (0,50) (50,50) 
Error al dibujar hamburguesa: operands could not be broadcast together with shapes (0,50) (50,50) 
Error al dibujar hamburguesa: operands could not be broadcast together with shapes (0,50) (50,50) 
Error al dibujar hamburguesa: operands could not be broadcast together with shapes (0,50) (50,50) 
Error al dibujar hamburguesa: operands could not be broadcast together with shapes (0,50) (50,50) 
Error al dibujar hamburguesa: operands could not be broadcast together with shapes (0,50) (50,50) 
Error al dibujar hamburguesa: operands could not be broadcast together with shapes (0,50) (50,50) 
Error al dibujar hamburguesa: operands could not be broadcast together with shapes (0,50) (50,50) 
Error al dibujar hamburguesa: operands could not be broadcast togeth